In [1]:
import pandas as pd

# Load the CSV file into a DataFrame
df_bundesliga_recencyfeatures = pd.read_csv('df_bundesliga_recencyfeatures.csv')

In [13]:
# Set pandas to display all columns
pd.set_option('display.max_columns', None)

In [14]:
df_bundesliga_recencyfeatures

,Unnamed: 0,matchday,date,home_team,final_result,away_team,home_goals,away_goals,winner,goal_difference,season_start_year,year,total_goals,home_recent_goals_scored_n3,home_recent_goals_conceded_n3,away_recent_goals_scored_n3,away_recent_goals_conceded_n3,home_recent_goals_scored_n5,home_recent_goals_conceded_n5,away_recent_goals_scored_n5,away_recent_goals_conceded_n5,home_recent_goals_scored_n10,home_recent_goals_conceded_n10,away_recent_goals_scored_n10,away_recent_goals_conceded_n10,previous_win,home_team_position,away_team_position,strength_difference,month,home_previous_position,away_previous_position,home_prev_season_goals_scored_n5,home_prev_season_goals_conceded_n5,away_prev_season_goals_scored_n5,away_prev_season_goals_conceded_n5,VfB Leipzig_previous_position,FC Ingolstadt 04_previous_position,Borussia Neunkirchen_previous_position,Energie Cottbus_previous_position,Alemannia Aachen_previous_position,Fortuna Köln_previous_position,Hannover 96_previous_position,SC Freiburg_previous_position,Arminia Bielefeld_previous_position,SC Tasmania 1900 Berlin_previous_position,VfL Wolfsburg_previous_position,SpVgg Unterhaching_previous_position,Wuppertaler SV_previous_position,VfL Bochum_previous_position,Blau-Weiß 90 Berlin_previous_position,Bayer 04 Leverkusen_previous_position,Fortuna Düsseldorf_previous_position,SV Darmstadt 98_previous_position,SV Waldhof Mannheim_previous_position,SpVgg Greuther Fürth_previous_position,Rot-Weiß Oberhausen_previous_position,FC Augsburg_previous_position,SSV Ulm 1846_previous_position,Rot-Weiss Essen_previous_position,Bor. Mönchengladbach_previous_position,KFC Uerdingen_previous_position,Stuttgarter Kickers_previous_position,RB Leipzig_previous_position,FC 08 Homburg_previous_position,Dynamo Dresden_previous_position,1899 Hoffenheim_previous_position,SC Paderborn 07_previous_position,SG Wattenscheid 09_previous_position,TSG 1899 Hoffenheim_previous_position,Kickers Offenbach_previous_position,Hansa Rostock_previous_position,1. FC Union Berlin_previous_position,Bayern München_previous_position,FC St. Pauli_previous_position,1. FC Heidenheim_previous_position,1. FSV Mainz 05_previous_position,Tennis Borussia Berlin_previous_position,Borussia Dortmund_previous_position,Borussia Dortmund_prev_season_goals_scored_n5,Borussia Dortmund_prev_season_goals_conceded_n5,Hamburger SV_previous_position,Hamburger SV_prev_season_goals_scored_n5,Hamburger SV_prev_season_goals_conceded_n5,1. FC Kaiserslautern_previous_position,1. FC Kaiserslautern_prev_season_goals_scored_n5,1. FC Kaiserslautern_prev_season_goals_conceded_n5,1. FC Köln_previous_position,1. FC Köln_prev_season_goals_scored_n5,1. FC Köln_prev_season_goals_conceded_n5,1. FC Nürnberg_previous_position,1. FC Nürnberg_prev_season_goals_scored_n5,1. FC Nürnberg_prev_season_goals_conceded_n5,Eintracht Frankfurt_previous_position,Eintracht Frankfurt_prev_season_goals_scored_n5,Eintracht Frankfurt_prev_season_goals_conceded_n5,Eintracht Braunschweig_previous_position,Eintracht Braunschweig_prev_season_goals_scored_n5,Eintracht Braunschweig_prev_season_goals_conceded_n5,Karlsruher SC_previous_position,Karlsruher SC_prev_season_goals_scored_n5,Karlsruher SC_prev_season_goals_conceded_n5,FC Schalke 04_previous_position,FC Schalke 04_prev_season_goals_scored_n5,FC Schalke 04_prev_season_goals_conceded_n5,Werder Bremen_previous_position,Werder Bremen_prev_season_goals_scored_n5,Werder Bremen_prev_season_goals_conceded_n5,MSV Duisburg_previous_position,MSV Duisburg_prev_season_goals_scored_n5,MSV Duisburg_prev_season_goals_conceded_n5,TSV 1860 München_previous_position,TSV 1860 München_prev_season_goals_scored_n5,TSV 1860 München_prev_season_goals_conceded_n5,Hertha BSC_previous_position,Hertha BSC_prev_season_goals_scored_n5,Hertha BSC_prev_season_goals_conceded_n5,VfB Stuttgart_previous_position,VfB Stuttgart_prev_season_goals_scored_n5,VfB Stuttgart_prev_season_goals_conceded_n5,Borussia Neunkirchen_prev_season_goals_scored_n5,Borussia Neunkirchen_prev_se

Plot graph to get an understanding of how what a typically win-,loss-, draws-streak looks like 

In [4]:
import plotly.express as px

team_streaks = {'team': [], 'streak_length': [], 'streak_type': []}

# Generate streaks for each team
for team in pd.concat([df_bundesliga_recencyfeatures['home_team'], df_bundesliga_recencyfeatures['away_team']]).unique():
    team_matches = df_bundesliga_recencyfeatures[(df_bundesliga_recencyfeatures['home_team'] == team) | (df_bundesliga_recencyfeatures['away_team'] == team)]
    team_matches = team_matches.sort_values('year')

    current_streak_type = None
    current_streak_length = 0

    for _, row in team_matches.iterrows():
        result = (
            'Win' if (row['winner'] == 0 and row['home_team'] == team) or (row['winner'] == 2 and row['away_team'] == team)
            else 'Draw' if row['winner'] == 1
            else 'Loss'
        )

        if result == current_streak_type:
            current_streak_length += 1
        else:
            if current_streak_type is not None:
                team_streaks['team'].append(team)
                team_streaks['streak_length'].append(current_streak_length)
                team_streaks['streak_type'].append(current_streak_type)

            current_streak_type = result
            current_streak_length = 1

    if current_streak_length > 0:
        team_streaks['team'].append(team)
        team_streaks['streak_length'].append(current_streak_length)
        team_streaks['streak_type'].append(current_streak_type)

streaks_df = pd.DataFrame(team_streaks)

plotly_fig = px.histogram(
    streaks_df, 
    x='streak_length', 
    color='streak_type', 
    barmode='stack',
    nbins=30,
    labels={'streak_length': 'Streak Length', 'count': 'Frequency'},
    title='Distribution of Streak Lengths in Bundesliga',
    color_discrete_sequence=px.colors.qualitative.Set2
)

plotly_fig.update_layout(
    xaxis_title='Streak Length',
    yaxis_title='Frequency',
    bargap=0.1,
    width=800,
    height=500
)

plotly_fig.show()

Calculate_recency_features for n matches

In [5]:
def calculate_recency_features(df, n):
 
    df['home_recent_goals_scored'] = 0
    df['home_recent_goals_conceded'] = 0
    df['away_recent_goals_scored'] = 0
    df['away_recent_goals_conceded'] = 0
    df['home_team_position'] = np.nan
    df['away_team_position'] = np.nan


   # Loop through each season
    for season in df['season_start_year'].unique():
        # Filter matches for the current season
        season_matches = df[df['season_start_year'] == season]

        # Initialize points table
        teams_in_season = pd.concat([season_matches['home_team'], season_matches['away_team']]).unique()
        points_table = {team: 0 for team in teams_in_season}
        league_table = pd.DataFrame(columns=['team', 'points', 'position'])

        # Loop through each match in the season
        for index, row in season_matches.iterrows():
            # Update points based on match outcome
            if row['winner'] == 0:  # Home win
                points_table[row['home_team']] += 3
            elif row['winner'] == 2:  # Away win
                points_table[row['away_team']] += 3
            else:  # Draw
                points_table[row['home_team']] += 1
                points_table[row['away_team']] += 1

            # Update the league table with current points
            league_table = pd.DataFrame(list(points_table.items()), columns=['team', 'points'])
            league_table = league_table.sort_values(by='points', ascending=False).reset_index(drop=True)
            league_table['position'] = league_table.index + 1

            # Assign current positions to the dataframe
            if row['home_team'] in league_table['team'].values:
                df.loc[index, 'home_team_position'] = league_table[league_table['team'] == row['home_team']]['position'].values[0]
            if row['away_team'] in league_table['team'].values:
                df.loc[index, 'away_team_position'] = league_table[league_table['team'] == row['away_team']]['position'].values[0]

            # Calculate recent goals scored and conceded for home and away matches
            home_matches = season_matches[season_matches['home_team'] == row['home_team']]
            away_matches = season_matches[season_matches['away_team'] == row['away_team']]

            if len(home_matches) > 1:
                df.loc[index, 'home_recent_goals_scored_n5'] = home_matches['home_goals'].rolling(window=n, min_periods=1).sum().shift(1).iloc[-1]
                df.loc[index, 'home_recent_goals_conceded_n5'] = home_matches['away_goals'].rolling(window=n, min_periods=1).sum().shift(1).iloc[-1]
            if len(away_matches) > 1:
                df.loc[index, 'away_recent_goals_scored_n5'] = away_matches['away_goals'].rolling(window=n, min_periods=1).sum().shift(1).iloc[-1]
                df.loc[index, 'away_recent_goals_conceded_n5'] = away_matches['home_goals'].rolling(window=n, min_periods=1).sum().shift(1).iloc[-1]

    # Calculate the strength difference
    df['strength_difference'] = df['home_team_position'] - df['away_team_position']
    df.fillna(0, inplace=True)
    
    return df


Run three models for 3 different values of n (3,5,10); Logistic Regression, Knn, Gradient Boosting


In [6]:
#Logistic Regression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

def run_logistic_regression(df, n):
    
    df = calculate_recency_features(df, n)

    features = [
        'home_recent_goals_scored', 'home_recent_goals_conceded',
        'away_recent_goals_scored', 'away_recent_goals_conceded',
        'previous_win', 'home_team_position', 
        'away_team_position', 'strength_difference', 'month'
    ]

    # Verify that the features exist in the DataFrame
    if not all(feature in df.columns for feature in features):
        raise ValueError(f"One or more of the required features are missing in the DataFrame. Available columns are: {df.columns.tolist()}")

    X = df[features]
    y = df['winner']

    # Drop rows with missing values in X or y
    X = X.dropna()
    y = y.loc[X.index] 

    # Reset index to avoid index misalignment issues
    X = X.reset_index(drop=True)
    y = y.reset_index(drop=True)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # logistic regression model
    model = LogisticRegression(max_iter=10000, random_state=42)
    model.fit(X_train_scaled, y_train)

    y_pred = model.predict(X_test_scaled)

    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy for n={n}: {accuracy:.2f}")

    print(f"Classification Report for n={n}:\n", classification_report(y_test, y_pred))

    return model

Plot feature importance for logistic regression model

In [7]:
import numpy as np
import matplotlib.pyplot as plt

def plot_feature_importance(model, features):
  
    coefficients = model.coef_[0]  
    feature_importance = np.abs(coefficients)  
    feature_importance, features = zip(*sorted(zip(feature_importance, features), reverse=True)) 

    
    plt.figure(figsize=(10, 6))
    plt.barh(features, feature_importance, color='skyblue')
    plt.xlabel('Feature Importance')
    plt.ylabel('Features')
    plt.title('Feature Importance from Logistic Regression')
    plt.gca().invert_yaxis() 
    plt.show()

print("\nFeature Importance for n=3:")
plot_feature_importance(log_reg_model_n3, [
    'home_recent_goals_scored', 'home_recent_goals_conceded',
    'away_recent_goals_scored', 'away_recent_goals_conceded',
    'previous_win', 'home_team_position', 
    'away_team_position', 'strength_difference', 'month'
])

print("\nFeature Importance for n=5:")
plot_feature_importance(log_reg_model_n5, [
    'home_recent_goals_scored', 'home_recent_goals_conceded',
    'away_recent_goals_scored', 'away_recent_goals_conceded',
    'previous_win', 'home_team_position', 
    'away_team_position', 'strength_difference', 'month'
])

print("\nFeature Importance for n=10:")
plot_feature_importance(log_reg_model_n10, [
    'home_recent_goals_scored', 'home_recent_goals_conceded',
    'away_recent_goals_scored', 'away_recent_goals_conceded',
    'previous_win', 'home_team_position', 
    'away_team_position', 'strength_difference', 'month'
])


Feature Importance for n=3:


NameError: name 'log_reg_model_n3' is not defined

In [8]:
#KNN 

from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score


def run_knn(df, n):
   
    df = calculate_recency_features(df, n)

    features = [
        'home_recent_goals_scored', 'home_recent_goals_conceded',
        'away_recent_goals_scored', 'away_recent_goals_conceded', 
        'previous_win', 'home_team_position', 
        'away_team_position', 'strength_difference', 'month'
    ]

    X = df[features]
    y = df['winner']

    X = X.dropna()
    y = y.loc[X.index] 

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # K-NN model
    knn = KNeighborsClassifier()

    param_grid = {'n_neighbors': [3, 5, 7, 10]}

    grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='accuracy')
    grid_search.fit(X_train_scaled, y_train)

    best_k = grid_search.best_params_['n_neighbors']
    print(f"Best k for n={n}: {best_k}")

    best_knn_model = grid_search.best_estimator_

    y_pred = best_knn_model.predict(X_test_scaled)

    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy for n={n}: {accuracy:.2f}")

    print(f"Classification Report for n={n}:\n", classification_report(y_test, y_pred))

    cv_scores = cross_val_score(best_knn_model, X_train_scaled, y_train, cv=5)
    print(f"Cross-validated accuracy for n={n}: {cv_scores.mean():.2f} ± {cv_scores.std():.2f}")

    return best_knn_model


In [9]:
# Run the k-NN model for different n values
print("\nResults for k-NN, n=3:")
knn_model_n3 = run_knn(df_bundesliga_recencyfeatures.copy(), 3)

print("\nResults for k-NN, n=5:")
knn_model_n5 = run_knn(df_bundesliga_recencyfeatures.copy(), 5)

print("\nResults for k-NN, n=10:")
knn_model_n10 = run_knn(df_bundesliga_recencyfeatures.copy(), 10)



Results for k-NN, n=3:
Best k for n=3: 10
Accuracy for n=3: 0.56
Classification Report for n=3:
               precision    recall  f1-score   support

           0       0.62      0.80      0.70      1883
           1       0.35      0.20      0.25       950
           2       0.49      0.42      0.45       889

    accuracy                           0.56      3722
   macro avg       0.49      0.47      0.47      3722
weighted avg       0.52      0.56      0.53      3722

Cross-validated accuracy for n=3: 0.55 ± 0.01

Results for k-NN, n=5:
Best k for n=5: 10
Accuracy for n=5: 0.56
Classification Report for n=5:
               precision    recall  f1-score   support

           0       0.62      0.80      0.70      1883
           1       0.35      0.20      0.25       950
           2       0.49      0.42      0.45       889

    accuracy                           0.56      3722
   macro avg       0.49      0.47      0.47      3722
weighted avg       0.52      0.56      0.53      37

In [19]:
#Gradient Boosting 

from sklearn.ensemble import GradientBoostingClassifier

def run_gradient_boosting(df, n):
   
    df = calculate_recency_features(df, n)

 
    features = [
        'home_recent_goals_scored', 
        'home_recent_goals_conceded', 
        'away_recent_goals_scored', 
        'away_recent_goals_conceded', 
        'previous_win', 
        'home_team_position', 
        'away_team_position', 
        'strength_difference',  
        'month' 
    ]

    X = df[features]
    y = df['winner'] 

    X = X.dropna()
    y = y.loc[X.index] 

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # GBDT model 
    gbdt = GradientBoostingClassifier(random_state=42)

    param_grid = {
        'n_estimators': [100],
        'learning_rate': [0.1],
        'max_depth': [3],
        'subsample': [0.8],
        'min_samples_split': [2],
        'min_samples_leaf': [1]
    }

    grid_search = GridSearchCV(gbdt, param_grid, cv=5, scoring='accuracy')

    grid_search.fit(X_train, y_train)

    best_params = grid_search.best_params_
    print(f"Best Parameters for n={n}: {best_params}")

    best_gbdt_model = grid_search.best_estimator_

    y_pred = best_gbdt_model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy for n={n}: {accuracy:.2f}")

    print(f"Classification Report for n={n}:\n", classification_report(y_test, y_pred))

    cv_scores = cross_val_score(best_gbdt_model, X_train, y_train, cv=5)
    print(f"Cross-validated accuracy for n={n}: {cv_scores.mean():.2f} ± {cv_scores.std():.2f}")

    return best_gbdt_model

In [20]:
print("\nResults for Gradient Boosting, n=3:")
gbdt_model_n3 = run_gradient_boosting(df_bundesliga_recencyfeatures.copy(), 3)

print("\nResults for Gradient Boosting, n=5:")
gbdt_model_n5 = run_gradient_boosting(df_bundesliga_recencyfeatures.copy(), 5)

print("\nResults for Gradient Boosting, n=9:")
gbdt_model_n9 = run_gradient_boosting(df_bundesliga_recencyfeatures.copy(), 9)


Results for Gradient Boosting, n=3:
Best Parameters for n=3: {'learning_rate': 0.1, 'max_depth': 3, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100, 'subsample': 0.8}
Accuracy for n=3: 0.59
Classification Report for n=3:
               precision    recall  f1-score   support

           0       0.63      0.86      0.73      1883
           1       0.45      0.09      0.15       950
           2       0.51      0.54      0.52       889

    accuracy                           0.59      3722
   macro avg       0.53      0.50      0.47      3722
weighted avg       0.55      0.59      0.53      3722

Cross-validated accuracy for n=3: 0.58 ± 0.01

Results for Gradient Boosting, n=5:
Best Parameters for n=5: {'learning_rate': 0.1, 'max_depth': 3, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100, 'subsample': 0.8}
Accuracy for n=5: 0.59
Classification Report for n=5:
               precision    recall  f1-score   support

           0       0.63      0.86 

In [52]:
def run_xgboost_model_reduced(df, n):
    
    df = calculate_recency_features(df, n)

    # Select only the most important features
    features = [
        'strength_difference', 'away_team_position', 
        'home_team_position', 'month',
        'strength_opposition', 'previous_win'
    ]

    # Verify that the features exist in the DataFrame
    if not all(feature in df.columns for feature in features):
        raise ValueError(f"One or more of the required features are missing in the DataFrame. Available columns are: {df.columns.tolist()}")

    X = df[features]
    y = df['winner']

    X = X.dropna()
    y = y.loc[X.index] 

    # Reset index to avoid index misalignment issues
    X = X.reset_index(drop=True)
    y = y.reset_index(drop=True)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # XGBoost model with hyperparameters from the study
    model = xgb.XGBClassifier(
        max_depth=6,
        learning_rate=0.1,
        n_estimators=100,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=0.5,
        reg_lambda=1.0,
        random_state=42,
        use_label_encoder=False,
        eval_metric='logloss'
    )
    
    model.fit(X_train_scaled, y_train)

    y_pred = model.predict(X_test_scaled)
    y_prob = model.predict_proba(X_test_scaled)

    accuracy = accuracy_score(y_test, y_pred)
    log_loss_value = log_loss(y_test, y_prob)
    rps_value = calculate_rps(y_test, y_prob)
    
    print(f"Accuracy for n={n}: {accuracy:.2f}")
    print(f"Log Loss for n={n}: {log_loss_value:.4f}")
    print(f"Ranked Probability Score (RPS) for n={n}: {rps_value:.4f}")

    print(f"Classification Report for n={n}:\n", classification_report(y_test, y_pred))

    return model


In [53]:
model = run_xgboost_model_reduced(df_bundesliga_recencyfeatures, n=9)

/Users/nilsweigeldt/.pyenv/versions/3.11.3/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning:

[12:07:01] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.




Accuracy for n=9: 0.60
Log Loss for n=9: 0.8691
Ranked Probability Score (RPS) for n=9: 0.1664
Classification Report for n=9:
               precision    recall  f1-score   support

           0       0.64      0.86      0.74      1883
           1       0.49      0.16      0.24       950
           2       0.53      0.53      0.53       889

    accuracy                           0.60      3722
   macro avg       0.55      0.52      0.50      3722
weighted avg       0.58      0.60      0.56      3722



Regression Model 

In [48]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

# Split 'final_result' into 'home_goals' and 'away_goals' (if not already done)
df_bundesliga_recencyfeatures[['home_goals_predicted', 'away_goals_predicted']] = df_bundesliga_recencyfeatures['final_result'].str.split('-', expand=True).astype(int)

# Features for prediction
features = [
    'home_recent_goals_scored_n5', 'home_recent_goals_conceded_n5',
    'away_recent_goals_scored_n5', 'away_recent_goals_conceded_n5',
    'home_team_position', 'away_team_position', 'strength_difference',
    'home_previous_position', 'away_previous_position'
]

X = df_bundesliga_recencyfeatures[features]

# Predicting Home Goals
y_home = df_bundesliga_recencyfeatures['home_goals_predicted']

# Predicting Away Goals
y_away = df_bundesliga_recencyfeatures['away_goals_predicted']

# Split the data for home goals prediction
X_train_home, X_test_home, y_train_home, y_test_home = train_test_split(X, y_home, test_size=0.2, random_state=42)

# Split the data for away goals prediction
X_train_away, X_test_away, y_train_away, y_test_away = train_test_split(X, y_away, test_size=0.2, random_state=42)

# Initialize the Gradient Boosting Regressor
gbr_home = GradientBoostingRegressor(random_state=42)
gbr_away = GradientBoostingRegressor(random_state=42)

# Grid search for hyperparameter tuning
param_grid = {
    'n_estimators': [200],
    'learning_rate': [0.1],
    'max_depth': [3],
    'subsample': [1.0]
}

# Home goals model
grid_search_home = GridSearchCV(gbr_home, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search_home.fit(X_train_home, y_train_home)

# Away goals model
grid_search_away = GridSearchCV(gbr_away, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search_away.fit(X_train_away, y_train_away)

# Best models
best_gbr_home = grid_search_home.best_estimator_
best_gbr_away = grid_search_away.best_estimator_

# Predict goals
y_pred_home = best_gbr_home.predict(X_test_home)
y_pred_away = best_gbr_away.predict(X_test_away)

# Evaluate the models
mse_home = mean_squared_error(y_test_home, y_pred_home)
mse_away = mean_squared_error(y_test_away, y_pred_away)

print(f"Mean Squared Error for Home Goals: {mse_home:.2f}")
print(f"Mean Squared Error for Away Goals: {mse_away:.2f}")

# If you want to see a sample of predictions vs actuals:
results_df = pd.DataFrame({
    'Actual Home Goals': y_test_home,
    'Predicted Home Goals': y_pred_home,
    'Actual Away Goals': y_test_away,
    'Predicted Away Goals': y_pred_away
})

print(results_df.head(10))



Mean Squared Error for Home Goals: 1.59
Mean Squared Error for Away Goals: 1.14
       Actual Home Goals  Predicted Home Goals  Actual Away Goals   
14677                  1              2.577895                  4  \
273                    3              2.552221                  2   
13476                  2              1.949903                  1   
6286                   3              4.208434                  2   
1599                   0              1.329954                  1   
9209                   4              1.701387                  0   
15796                  0              0.727173                  2   
8913                   4              2.696683                  1   
10294                  2              3.031558                  1   
14535                  1              1.064102                  3   

       Predicted Away Goals  
14677              0.815507  
273                0.693149  
13476              0.889390  
6286               0.808345  
1599      